### Platt Burges

&nbsp;
    
This script solves Platt-Burges model via regularized least squares with L2 norm penalty (Ridge Regression). Albeit the default solver in our repository is Expectation Maximization algorithm. Certainly there are merits to EM algorithm. 

1. You have a more detailed derivation from Stanford CS229 Autumn 2016 Problem Set 4 Problem 2. 
2. EM algorithm can give us the standard deviation of both intrinsic value and bias level whereas RLS only yields the mean of intrinsic value and bias level.

Despite all the malaises we list out, why are we doing RLS?

1. RLS is fast and straight forward. No need to get stuck in prior and posterior distribution.
2. RLS can work on incomplete matrix via L2 norm regularization. The penalty is on reviewer bias when some reviewers have not reviewed all the papers. Although the coefficient of penalty could be arbitrary as we cannot use cross validation to obtain the optima. Moreover, the adjacency matrix does not have to be binary to indicate a reviewer has reviewed a paper. The adjacency matrix can be modified into a confidence matrix where each element is within [0,1].

Assuming $P$ papers are submitted to the conference and $R$ reviewers in the committee mark the score of these papers, each paper will be given $R$ different scores by all the reviewers. Therefore, the score of a paper given by a reviewer, denoted as $x$, can be decomposed into the linear combination of three components – the underlying intrinsic value $y$, the reviewer bias $z$ and some random disturbance $\epsilon$. $x$, $y$ and $z$ independently follow different Gaussian distributions.

$$ y^{(pr)} \sim \mathcal{N} (\mu_p,\sigma_p^2)$$

$$ z^{(pr)} \sim \mathcal{N} (\nu_r,\tau_r^2)$$

$$ x^{(pr)}|y^{(pr)},z^{(pr)} \sim \mathcal{N} (y^{(pr)}+z^{(pr)},\sigma^2)$$

RLS solves Platt-Burges model by minimizing the loss function $\mathcal{L}$. $\lambda$ denotes the coefficient of L2 penalty. $A^{(pr)}$ denotes the adjacency matrix (or confidence matrix) where not every reviewer reviews all the papers. When the adjacency matrix is filled with ones, $\lambda$ should be zero. 

$$ \mathcal{L}=\frac {1}{2} \sum_{p=1}^P\sum_{r=1}^R A^{(pr)} (x^{(pr)}-\mu_p-\nu_r)^2+\frac {1}{2} \sum_{r=1}^R \lambda \nu_r^2$$

&nbsp;

For EM algorithm, plz check the below

https://github.com/je-suis-tm/machine-learning/blob/master/Wisdom%20of%20Crowds%20project/platt%20burges.ipynb

Reference to the original paper

https://github.com/je-suis-tm/machine-learning/blob/master/Wisdom%20of%20Crowds%20project/Regularized%20Least%20Squares%20to%20Remove%20Reviewer%20Bias.pdf

Reference to Hong Ge's paper

http://mlg.eng.cam.ac.uk/hong/unpublished/nips-review-model.pdf

Neil Lawrence's personal blog

https://inverseprobability.com/2014/08/02/reviewer-calibration-for-nips

Neil Lawrence's jupyter notebook

https://github.com/lawrennd/conference

Others' jupyter notebook

https://github.com/leonidk/reviewers

In [1]:
import matplotlib.pyplot as plt
import os
os.chdir('K:/ecole/github/televerser/wisdom of crowds')
import numpy as np
import pandas as pd
import scipy.optimize

In [2]:
#raise error when zero is encountered in logarithm
np.seterr(divide='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Functions

In [3]:
#compute rls loss function
def loss_function(x0,data,lambda_,adjacency_matrix):
    
    #unpack
    intrinsic_value=x0[:data.shape[1]]
    bias_level=x0[data.shape[1]:]

    #convert intrinsic value and bias lvl into matrix
    miu_p=np.repeat(np.array(intrinsic_value).reshape(1,-1),
                    data.shape[0],axis=0)
    nu_r=np.repeat(np.array(bias_level).reshape(-1,1),
                   data.shape[1],axis=1)

    #compute loss function
    rls_loss=np.multiply(np.square(data-miu_p-nu_r),
       adjacency_matrix).sum()/2+lambda_*np.square(bias_level).sum()/2
    
    return rls_loss

In [4]:
#using rls to solve platt burges
def regularized_least_square(X,lambda_=0.5,
                             adjacency_matrix=[],**kwargs):
    
    #if not defined
    #use complete matrix
    if len(adjacency_matrix)==0:
        adjacency_matrix=np.ones(X.shape)

    #pack
    miu_init=X.mean(axis=0).ravel().tolist()[0]
    nu_init=X.mean(axis=1).ravel().tolist()[0]
    x0=miu_init+nu_init

    #rls
    result=scipy.optimize.minimize(loss_function,x0,
                                   args=(X,lambda_,adjacency_matrix),
                                   **kwargs
                                  )

    if result['success']:

        #unpack
        intrinsic_value=result['x'][:X.shape[1]]
        bias_level=result['x'][X.shape[1]:]

        return intrinsic_value,bias_level

### ETL

In [5]:
#read data
y0matrix2019=pd.read_csv('y0matrix2019.csv')

y1matrix2020=pd.read_csv('y1matrix2020.csv')

monthly=pd.read_csv('monthly.csv')

annual=pd.read_csv('annual.csv')

In [6]:
#set index
y0matrix2019.set_index('Source Name',inplace=True)

y1matrix2020.set_index('Source Name',inplace=True)

monthly.set_index('Date',inplace=True)
monthly.index=pd.to_datetime(monthly.index)
monthly.columns=y0matrix2019.columns

annual=annual.pivot(index='Date',
                    columns='Name',values='Value')
annual.index=pd.to_datetime(annual.index)
annual.columns=y0matrix2019.columns

In [7]:
#normalize forecast by pct return
y0_mat_nor=np.mat(
    np.divide(y0matrix2019,
              monthly['2019-08-31':'2019-08-31'])-1)
y1_mat_nor=np.mat(
    np.divide(y1matrix2020,
              monthly['2019-08-31':'2019-08-31'])-1)

### Run Model

In [8]:
#current year outlook
intrinsic_value,bias_level=regularized_least_square(
    y0_mat_nor,lambda_=0,)

print(bias_level)

#comparison with result from em
print([0.02019064,0.02024495,0.00609068,-0.01649482,-0.00426001,
0.00847965,-0.00821187,0.01734934])

[ 0.03496034  0.03416314  0.02020078 -0.00271194  0.00968679  0.02259967
  0.00561574  0.03175916]
[0.02019064, 0.02024495, 0.00609068, -0.01649482, -0.00426001, 0.00847965, -0.00821187, 0.01734934]


In [9]:
#one year ahead outlook
intrinsic_value,bias_level=regularized_least_square(
    y1_mat_nor,lambda_=0,)

print(bias_level)

#comparison with result from em
print([0.01352132,0.02981012,0.01582776,-0.00265233,-0.01151194,
-0.01075301,-0.01477036,0.12330401])

[ 0.02044496  0.03448161  0.02317991  0.00170572 -0.00614319 -0.00719274
 -0.00882933  0.13096969]
[0.01352132, 0.02981012, 0.01582776, -0.00265233, -0.01151194, -0.01075301, -0.01477036, 0.12330401]


In [10]:
#demo on incomplete matrix
intrinsic_value,bias_level=regularized_least_square(
    y1_mat_nor,lambda_=0.5,
    adjacency_matrix=np.random.choice([0,1],size=y1_mat_nor.shape))

print(bias_level)

[-2.29570274e-02  9.72272315e-03  2.67925635e-02 -2.04535244e-02
 -3.98935081e-02 -4.15196790e-02 -8.67932470e-05  8.83924600e-02]


In [11]:
#demo on confidence interval
intrinsic_value,bias_level=regularized_least_square(
    y1_mat_nor,lambda_=0.5,
    adjacency_matrix=np.random.uniform(size=y1_mat_nor.shape))

print(bias_level)

[-0.00532965  0.00653886  0.01550822 -0.02454352 -0.03179216 -0.0208043
 -0.02790948  0.08833376]
